In [1]:
# Import all necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import mplfinance as mpf
import plotly.graph_objects as go
import pandas as pd
import cv2
import matplotlib.image as mpimg
import os
from PIL import Image

In [2]:
# Run locally
folder_link = ".."

In [17]:
train_data = []
train_output_arr = []
for coin in ["ada", "btc", "doge", "dot", "eth", "luna", "matic", "shib", "sol", "xrp"]:
    train_data.append(np.memmap(f"{folder_link}/image_data/{coin}usdt/training_data.dat", dtype = np.uint8, mode = 'r').reshape((-1, 54, 72)))
    train_data.append(np.memmap(f"{folder_link}/image_data/{coin}usdt/validation_data.dat", dtype = np.uint8, mode = 'r').reshape((-1, 54, 72)))
    train_output_arr.append(pd.read_csv(f"{folder_link}/image_data/{coin}usdt/training_output.csv"))
    train_output_arr.append(pd.read_csv(f"{folder_link}/image_data/{coin}usdt/validation_output.csv"))

train_data = np.concatenate(train_data)
print(train_data.shape)

(280437, 54, 72)


In [4]:
# Check the type
type(train_data)

numpy.ndarray

In [6]:
# Save train_data first
np.savez_compressed("train_data", train_data, allow_pickle = False)

In [18]:
def normalize(df):
    df["RollingMaxLR"] = df["LogReturns"].rolling(window = 24).apply(lambda x: np.max(x) - np.min(x))
    df["RollingMaxLR"] = df["RollingMaxLR"].fillna(df.loc[23, "RollingMaxLR"])
    df["NormalizedLogReturns"] = df["LogReturns"] / df["RollingMaxLR"]

for i in range(len(train_output_arr)):
    normalize(train_output_arr[i])

In [22]:
train_output = pd.concat(train_output_arr)
train_output = train_output.drop(["Volume", "Close time", "Quote asset volume", 'Number of trades', "Taker buy base asset volume", "Taker buy quote asset volume"], axis = 1)
train_output

,Open time,High,Low,LogReturns,RollingMaxLR,NormalizedLogReturns
0,1609545600000,0.17572,0.17028,-0.019493,0.094469,-0.206345
1,1609549200000,0.17423,0.17016,0.010832,0.094469,0.114658
2,1609552800000,0.17474,0.17323,0.004025,0.094469,0.042603
3,1609556400000,0.17618,0.17412,0.004180,0.094469,0.044247
4,1609560000000,0.17516,0.17313,-0.004065,0.094469,-0.043032
...,...,...,...,...,...,...
2154,1711908000000,0.63010,0.62510,0.002394,0.009461,0.253058
2155,1711911600000,0.62940,0.62480,0.002229,0.009461,0.235642
2156,1711915200000,0.62950,0.62690,-0.002070,0.009461,-0.218793
2157,1711918800000,0.62780,0.62500,-0.000638,0.008814,-0.072357


In [23]:
# Save to CSV
train_output.to_csv("train_output.csv", index = False)

In [24]:
test_data = []
test_output_arr = []
for coin in ["ada", "btc", "doge", "dot", "eth", "luna", "matic", "shib", "sol", "xrp"] + ["bonk", "floki", "pepe", "wif"]:
    test_data.append(np.memmap(f"{folder_link}/image_data/{coin}usdt/testing_data.dat", dtype = np.uint8, mode = 'r').reshape((-1, 54, 72)))
    test_output_arr.append(pd.read_csv(f"{folder_link}/image_data/{coin}usdt/testing_output.csv"))

test_data = np.concatenate(test_data)
print(test_data.shape)

(70297, 54, 72)


In [13]:
np.savez_compressed("test_data", test_data, allow_pickle = False)

In [25]:
for i in range(len(test_output_arr)):
    normalize(test_output_arr[i])

test_output = pd.concat(test_output_arr)
test_output = test_output.drop(["Volume", "Close time", "Quote asset volume", 'Number of trades', "Taker buy base asset volume", "Taker buy quote asset volume"], axis = 1)
test_output

,Open time,High,Low,LogReturns,RollingMaxLR,NormalizedLogReturns
0,1712016000000,0.6214,0.6150,-0.006622,0.042310,-0.156513
1,1712019600000,0.6199,0.6128,0.001943,0.042310,0.045916
2,1712023200000,0.6184,0.5856,-0.032046,0.042310,-0.757419
3,1712026800000,0.6024,0.5941,-0.005190,0.042310,-0.122678
4,1712030400000,0.5999,0.5944,-0.000168,0.042310,-0.003968
...,...,...,...,...,...,...
5106,1730397600000,NaN,NaN,-0.015255,0.043069,-0.354195
5107,1730401200000,NaN,NaN,-0.016758,0.043069,-0.389091
5108,1730404800000,NaN,NaN,0.006317,0.041514,0.152169
5109,1730408400000,NaN,NaN,0.003771,0.041514,0.090842


In [16]:
# Save to CSV
test_output.to_csv("test_output.csv", index = False)